<a href="https://colab.research.google.com/github/Raghava1946/FMML-22B21A4455/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

The accuracy of the validation set can be affected by the percentage of data allocated to it during model training. Here's what happens when you increase or decrease the percentage of the validation set:

1. *Increase Validation Set Percentage:*
   - *Pros:* A larger validation set can provide a more representative sample of your data for model evaluation. It may lead to a more accurate estimate of your model's performance on unseen data.
   - *Cons:* If you allocate too much data to the validation set, you may have less data available for training the model. This can result in underfitting, where the model doesn't learn well from the training data due to limited samples.

2. *Decrease Validation Set Percentage:*
   - *Pros:* More data available for training can lead to a model that fits the training data better and potentially achieves higher training accuracy.
   - *Cons:* With a smaller validation set, you might not get a reliable estimate of how well your model generalizes to new, unseen data. This can lead to overfitting, where the model performs well on the training data but poorly on new data.

Balancing the percentage of data between training and validation sets is a crucial aspect of machine learning model development. It involves finding a trade-off that ensures your model learns effectively while still providing a good estimate of its generalization performance. Techniques like cross-validation can also be used to mitigate some of these issues by repeatedly partitioning the data into training and validation sets to get a more robust evaluation.

The size of the training and validation sets can significantly affect how well you can predict the accuracy on the test set using the validation set. Here are some key points to consider:

1. *Smaller Training Set, Larger Validation Set:*
   - If you have a smaller training set and a larger validation set, your model may not learn the underlying patterns in the data effectively. This can lead to underfitting because the model lacks exposure to diverse examples during training.

2. *Larger Training Set, Smaller Validation Set:*
   - Conversely, if you have a larger training set and a smaller validation set, your model may overfit to the training data, as it has too much capacity to learn from a limited set of validation examples. This could result in an overly optimistic estimate of your model's performance.

3. *Balanced Split:*
   - Ideally, you should aim for a balanced split where you have enough data in the training set to capture the underlying patterns in your data, and a sizable validation set to evaluate your model's performance accurately. A common split is 70-80% for training and 20-30% for validation.

4. *Cross-Validation:*
   - To mitigate the impact of the initial split, you can also use techniques like k-fold cross-validation. This involves dividing the data into k subsets, training the model k times with different subsets as the validation set each time, and averaging the results to get a more robust estimate of performance.

5. *Randomness and Variability:*
   - It's important to recognize that there can be randomness and variability in the data. A different random split of the data can yield different results. Cross-validation helps reduce this variability.

6. *Hyperparameter Tuning:*
   - The choice of hyperparameters can also impact the model's performance on the validation set. You may need to perform hyperparameter tuning using the validation set to optimize your model's performance.

In summary, the size and balance of the training and validation sets are crucial factors in assessing your model's performance. Striking the right balance helps you make a more reliable prediction of how well your model will generalize to the test set.

The percentage you reserve for the validation set in machine learning depends on various factors, including the size of your dataset, the complexity of your model, and the specific problem you're trying to solve. There's no one-size-fits-all answer, but a common practice is to use a 70-30 or 80-20 split for training and validation data, respectively. However, in cases with very large datasets, you might use a smaller validation set, like 10% or even less.

The key is to strike a balance between having enough data to train your model effectively while reserving enough for validation to assess its performance accurately. Cross-validation techniques like k-fold cross-validation can also help if you have limited data. Experimentation and adjusting the split percentage based on your specific project's requirements and constraints can be a good approach.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


Yes, averaging validation accuracy across multiple splits, typically in a technique called k-fold cross-validation, can provide more consistent and reliable results. It helps reduce the impact of random variations in the data split and provides a better estimate of the model's performance. This is especially useful when dealing with limited data, as it gives a more robust assessment of your model's generalization ability.
The accuracy of a test estimate depends on various factors, such as the quality and quantity of data, the complexity of the model, and the evaluation metrics used. Generally, larger and more diverse datasets tend to provide more accurate estimates of test accuracy. Complex models may overfit to the training data, leading to less accurate estimates on the test data.

To improve the accuracy of test estimates, you can consider techniques like cross-validation, which helps assess model performance across multiple data subsets, or using more informative evaluation metrics beyond simple accuracy, such as precision, recall, F1-score, or ROC-AUC.

Ultimately, the accuracy of a test estimate is a result of careful data preparation, model selection, and evaluation methods.
The effect of the number of iterations on an estimate depends on the specific algorithm or method being used. In many cases, increasing the number of iterations can lead to a more accurate estimate, up to a point. However, there are diminishing returns, and at some point, additional iterations may not significantly improve the estimate while increasing computational cost.

For example, in numerical methods like the Monte Carlo method, more iterations generally lead to a more accurate estimate as the algorithm samples more data points. However, after a certain number of iterations, the improvement in accuracy may become negligible, and it might not be worth the computational resources.

In optimization algorithms like gradient descent, increasing the number of iterations can help in finding a better solution, but it also depends on other factors like the learning rate and the specific problem being solved.

So, whether you get a better estimate with higher iterations depends on the algorithm, problem, and the trade-off between computational cost and accuracy you are willing to make. It's often a balance between accuracy and efficiency.
Increasing the number of iterations can help to some extent in training with a smaller dataset, but it may not fully compensate for the limitations of a small dataset. Training with a very small dataset can lead to overfitting, where the model becomes too specialized to the limited examples it has seen, and may not generalize well to new, unseen data.

While increasing the number of iterations can improve the model's performance on the training data, it may not necessarily improve its ability to generalize to validation or test data. The model may still struggle with unseen examples or exhibit high variance.

To mitigate the issues of a small dataset, it's often better to explore other strategies, such as data augmentation, transfer learning, or using pre-trained models. These approaches can help leverage existing knowledge or generate more diverse training examples to improve model generalization, even with limited data.